In [9]:
import requests 
import json
import pandas as pd
import numpy as np

In [22]:
#드라이브에 올린 파일에 제가 '영화코드' 열을 추가했습니다.
#저는 일일이 영화코드를 찾아서 입력한다음, 엑셀파일을 다운로드하여 불러오기 했지만
#여러분들은 좀 더 나은 방법이 있으면 찾아서 해보는걸 추천해요.. 노가다 ㅠㅠ
movie = pd.read_csv("/Users/j-max/Documents/project_python/데이터 총합본.csv")
movie_code = movie["영화코드"]

#그대로 불러오면 float형태라 소숫점자리도 표기되고, 
#입력값으로 사용하려면 문자열로 바꿔야함
code = pd.Series(movie_code[0:len(movie)]).apply(int).apply(str)
code

0      20081672
1      20072152
2      20080046
3      20080736
4      20070382
         ...   
204    20199541
205    20193068
206    20183003
207    20203286
208    20194501
Name: 영화코드, Length: 209, dtype: object

In [23]:
#key값은 직접 발급받으신 값으로 변경해서 사용하셔야 합니다.
#하나의 key로는 하루에 불러올 수 있는 데이터 수가 한정되어있어서 개인 key를 사용하는게 좋습니다.

my_key = 'be95a18fd733d2992b2298a8c19cb765'
url = ' http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json'

#불러온 데이터를 저장할 빈 데이터프레임 생성
act_df = pd.DataFrame(columns = ['영화코드', '영화제목','출연배우'])

for ii in range(0, len(code)):
    data= {
        "key": my_key,
    "movieCd": code[ii]
    }
    
    res = requests.get(url, data)
    text = res.text
    
    d = json.loads(text)
    #json데이터는 계층별로 나뉘어져 있습니다, 아래에서 한번 확인해보세요
    #불러온 데이터 > movieInfoRsult > movieInfo
    x = d['movieInfoResult']['movieInfo']
    
    #> movieCd
    #> movidNm
    MvCd = x['movieCd']
    MvNm = x['movieNm']
    
    #변수 초기화
    actNm = ' ' 
    
    #actors > 다수의 목록이 존재, 숫자로 인덱싱 > 'peopleNm'
    for jj in range(0, len(x['actors'])):
        #배우의 이름은 문자열이므로 데이터프레임으로 다루기 편하도록 이어붙인다.
        actNm = actNm + "." + x['actors'][jj]['peopleNm']
    
    act_df.loc[ii] = [MvCd, MvNm, actNm]
    
act_df

KeyError: 'movieInfoResult'

In [7]:
#csv파일로 저장, 이름 바꾸셔서 사용하면 되고, 
#저장경로는 이 ipynb파일이 있는 폴더입니다.

act_df.to_csv("2008-2021actor.csv", mode='a', encoding='utf-8', index=False)